In [1]:
# Clear the R environment and set working directory
rm(list = ls())
setwd("/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters")
path <- "/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters"

In [2]:
# Install and load additional packages needed
# install.packages("spatstat")
library(did)
library(rio)
library(ggplot2)

Loading required package: spatstat.data

Loading required package: spatstat.geom

spatstat.geom 2.4-0

Loading required package: spatstat.random

spatstat.random 2.2-0

Loading required package: spatstat.core

Loading required package: nlme

Loading required package: rpart

spatstat.core 2.4-4

Loading required package: spatstat.linnet

spatstat.linnet 2.3-2


spatstat 2.3-4       (nickname: ‘Watch this space’) 
For an introduction to spatstat, type ‘beginner’ 




In [3]:
# read in the data set
btw <-import(sprintf("%s/data/btw_treat.csv", path))
summary(btw)

      AGS               GEN                 year          Land          
 Min.   : 1001000   Length:156154      Min.   :1994   Length:156154     
 1st Qu.: 6535012   Class :character   1st Qu.:1998   Class :character  
 Median : 8136082   Mode  :character   Median :2005   Mode  :character  
 Mean   : 8409026                      Mean   :2005                     
 3rd Qu.: 9779226                      3rd Qu.:2013                     
 Max.   :16077055                      Max.   :2017                     
                                                                        
    state_id          Kreis          Wahlkreis       post_1998     
 Min.   : 1.000   Min.   : 101.0   Min.   :  1.0   Min.   :0.0000  
 1st Qu.: 6.000   1st Qu.: 635.0   1st Qu.: 59.0   1st Qu.:0.0000  
 Median : 8.000   Median : 817.0   Median :195.0   Median :1.0000  
 Mean   : 8.141   Mean   : 867.8   Mean   :160.8   Mean   :0.7142  
 3rd Qu.: 9.000   3rd Qu.: 990.0   3rd Qu.:231.0   3rd Qu.:1.0000  
 Max.   

In [39]:
state_dummies <- dummify(btw$Land)

In [40]:
btw <- cbind(btw, state_dummies)

In [43]:
colnames(btw)

[1] "AGS"           "GEN"           "year"          "Land"         
 [5] "state_id"      "Kreis"         "Wahlkreis"     "post_1998"    
 [9] "post_2002"     "post_2005"     "post_2009"     "post_2013"    
[13] "first_vote"    "second_vote"   "Wähler"        "Gültig"       
[17] "Union"         "SPD"           "FDP"           "Linke"        
[21] "Grüne"         "Andere"        "treatment_05"  "treatment_09" 
[25] "treatment_13"  "treatment_30"  "treatment_60"  "treatment_100"
[29] "BB"            "BE"            "BW"            "BY"           
[33] "HB"            "HE"            "HH"            "MV"           
[37] "NI"            "NW"            "RP"            "SH"           
[41] "SL"            "SN"            "ST"            "TH"

In [4]:
# Filter for first and second vote
erst <- subset(btw, first_vote == 1)
zweit <- subset(btw, second_vote == 1)

In [6]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments <-  c('treatment_30', 'treatment_60', 'treatment_100')

for(party in parties){
  for(treatment in treatments){
    out <- att_gt(yname = party,
                  gname = treatment,
                  idname = 'AGS',
                  panel = TRUE,
                  tname = 'year',
                  xformla = ~ 1, # BE + BW + BY + HB + HE + HH + MV + NI +NW + RP  + SH + SL + SN + ST + TH, # C(Land),
                  data = erst,
                  est_method = 'reg',
                  anticipation = 0,
                  control_group = 'nevertreated',
                  clustervars = c('AGS'),
                  #bstrap = TRUE,
                  #cband = TRUE,
                  allow_unbalanced_panel = TRUE,
    )
    summary(out)
    #ggdid(out, ylim = c(floor(min(out$att - out$se * 2.345 - 1)), ceiling(max(out$att + out$se * 2.345 + 1))))
    #ggsave(sprintf('%s/figures/R/Group/%s_%s.png', path, party, treatment))
    es <- aggte(out, type = 'dynamic')
    summary(es)
    #ggdid(es, ylim = c(floor(min(es$att.egt - es$se.egt * 2.345 - 1)), ceiling(max(es$att.egt + es$se.egt * 2.345 + 1))))
    #ggsave(sprintf('%s/figures/R/ES/ES_%s_%s.png', path, party, treatment))
  }
}


Call:
att_gt(yname = party, tname = "year", idname = "AGS", gname = treatment, 
    xformla = ~1, data = erst, panel = TRUE, allow_unbalanced_panel = TRUE, 
    control_group = "nevertreated", anticipation = 0, clustervars = c("AGS"), 
    est_method = "reg")

Reference: Callaway, Brantly and Pedro H.C. Sant'Anna.  "Difference-in-Differences with Multiple Time Periods." Journal of Econometrics, Vol. 225, No. 2, pp. 200-230, 2021. <https://doi.org/10.1016/j.jeconom.2020.12.001>, <https://arxiv.org/abs/1803.09015> 

Group-Time Average Treatment Effects:
 Group Time ATT(g,t) Std. Error [95% Simult.  Conf. Band]  
  2005 1998  -0.5722     0.1744       -1.0924     -0.0520 *
  2005 2002  -1.6341     0.1529       -2.0903     -1.1779 *
  2005 2005   1.0115     0.1421        0.5876      1.4355 *
  2005 2009   3.0319     0.1845        2.4815      3.5824 *
  2005 2013   2.1855     0.2035        1.5784      2.7926 *
  2005 2017   5.2040     0.2373        4.4961      5.9120 *
  2010 1998   1.0759 